# IBM Applied Data Science Capstone Course by Coursera


Week-3 Part-1,2,3

* To Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name in the city of   Toronto.
* Get the geographical coordinates of the neighborhoods in Toronto.
* Explore and cluster the neighborhoods in Toronto.






# 1.  Importing Libraries

In [65]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


from bs4 import BeautifulSoup # library to parse HTML and XML documents

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# 2. Scrapping the wikipedia data into data frame using beautiful soup.

In [66]:
# send the GET request to retrive the data from wikipedia page consisting toronto neighbourhood details.

dataset = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [67]:
# Parsing the data from the html into a beautifulsoup object

soup = BeautifulSoup(dataset, 'html.parser')

In [68]:
# create three fields to store the data
postalCode = []
borough = []
neighborhood = []

# Now as the wikipedia page has been scrapped successfully we need to look        for the specific data such as the table of our interest.

   Hence here comes the use of BeautifulSoup to specifically retrieve the table.

# for each row of the table, find all the table data i.e for each row all columns's      data should be made available.

   
   



In [69]:
#Using the BeautifulSoup Command to append the data into the respective data fields.

for rows in soup.find('table').find_all('tr'):
    cellData=rows.find_all('td')
    if(len(cellData) > 0):
        postalCode.append(cellData[0].text)
        borough.append(cellData[1].text)
        neighborhood.append(cellData[2].text.rstrip('\n')) 
        
        
        

In [70]:
#Converting it into a data frame
toronto_DF=pd.DataFrame({"PostalCode":postalCode,
                        "Borough":borough,
                        "Neighborhood":neighborhood})

toronto_DF.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# 3.Dropping the borough values having 'Not assigned' values.

In [71]:
toronto_DF=toronto_DF[toronto_DF.Borough !="Not assigned"].reset_index(drop=True)
toronto_DF.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


# 4. Group neighborhoods in the same borough

In [72]:
toronto_grouped=toronto_DF.groupby(["PostalCode","Borough"],as_index=False).agg(lambda x:",".join(x))

toronto_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# 5. For Neighborhood="Not assigned", make the value the same as Borough

In [73]:
for index,row in toronto_grouped.iterrows():
    if row["Neighborhood"]=="Not assigned":
        row["Neighborhood"]=row["Borough"]
        
        
toronto_grouped.head()        

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# 6. Check whether it is the same as required by the question as per the sample data

In [74]:
# create a new test dataframe

column_names = ["PostalCode", "Borough", "Neighborhood"]

test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postalcode in test_list:
    test_df=test_df.append(toronto_grouped[toronto_grouped["PostalCode"]==postalcode],ignore_index=True)
    
test_df.head()    


,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Woodbine Gardens,Parkview Hill"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside


# 7. Printing the shape of the resulting dataset.

In [75]:
toronto_grouped.shape

(103, 3)

# 8. Getting the geospatial cooridinates by the given Coursera.

In [76]:
import os
os.chdir("E:/capstone/week 3")

coordinates=pd.read_csv("Geospatial_Coordinates.csv")


#Rename the column name Postal Code as PostalCode

coordinates.rename(columns={"Postal Code":"PostalCode"},inplace=True)

coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# 9. Merging the two tables so that latitudes and longitudes could be added.

In [159]:
toronto_grouped1=toronto_grouped.merge(coordinates,on="PostalCode",how="left")

toronto_grouped1.head()


,PostalCode,Borough,Neighborhoods,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


# 10. Finally, check to make sure the coordinates are added as required by the question given in coursera

In [79]:
columns_names =["PostalCode","Borough","Neighborhood","Latitude","Longitude"]

test_df=pd.DataFrame(columns=columns_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]


for postalcode in test_list:
    test_df=test_df.append(toronto_grouped[toronto_grouped["PostalCode"]==postalcode],ignore_index=True)
    
test_df  

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Maryvale,Wexford",43.750072,-79.295849
7,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.628947,-79.394420


# 11. Using the  geopy library to get the latitude and longitude values of Toronto city.

In [81]:
address="Toronto"

geolocator=Nominatim(user_agent="Foursquare_agent")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude

print("The latitude is :{} and the longitude is:{}".format(latitude,longitude))

The latitude is :43.653963 and the longitude is:-79.387207


# 12. Create a map of Toronto with neighborhoods.

In [86]:
toronto_map=folium.Map(location=[latitude,longitude],zoom_start=10)



#Adding the markers of neighborhoods by providing the latitude longitude of the toronto.

for lat,lng,borough,neighborhood in zip(toronto_grouped["Latitude"],toronto_grouped['Longitude'],toronto_grouped['Borough'],
                                       toronto_grouped['Neighborhood']):
    label= '{},{}'.format(neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lng],
         radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(toronto_map)
    
    
toronto_map    

# 13. Filter only boroughs that contain the word Toronto.

In [91]:
#filter the boroughs names having unique values

borough_names=list(toronto_grouped.Borough.unique())


#Filter those boroughs having only toronto.

borough_with_toronto=[]

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)

borough_with_toronto        
        

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [94]:
# create a new DataFrame with only boroughs that contain the word Toronto
toronto_grouped = toronto_grouped[toronto_grouped['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_grouped.shape)
toronto_grouped.head()

(38, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [97]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_grouped['Latitude'], toronto_grouped['Longitude'], toronto_grouped['Borough'], toronto_grouped['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

# 14. Use the Foursquare API to explore the neighborhoods.

In [98]:
# define Foursquare Credentials and Version
CLIENT_ID = 'UBKVQPHZ0N2M23PIWVP4UBKZQAK410PCIT2VABDOOV3G5MEG' # your Foursquare ID
CLIENT_SECRET = 'Y25V03BWK1PPRZRM4RZSJE55X1RWAOARBWWLWNB4VROPGREI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UBKVQPHZ0N2M23PIWVP4UBKZQAK410PCIT2VABDOOV3G5MEG
CLIENT_SECRET:Y25V03BWK1PPRZRM4RZSJE55X1RWAOARBWWLWNB4VROPGREI


# Now, let's get the top 100 venues that are within a radius of 500 meters.

In [113]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_grouped['Latitude'], toronto_grouped['Longitude'], toronto_grouped['PostalCode'], toronto_grouped['Borough'], toronto_grouped['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
           

In [117]:
# convert the venues list into a new DataFrame

venues_df=pd.DataFrame(venues)

#Define the columns names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(1707, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


## Now checking each postal code returning no. of venues.

In [119]:
venues_df.groupby(["PostalCode","Borough","Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4K,East Toronto,"The Danforth West,Riverdale",44,44,44,44,44,44
M4L,East Toronto,"The Beaches West,India Bazaar",20,20,20,20,20,20
M4M,East Toronto,Studio District,40,40,40,40,40,40
M4N,Central Toronto,Lawrence Park,3,3,3,3,3,3
M4P,Central Toronto,Davisville North,8,8,8,8,8,8
M4R,Central Toronto,North Toronto West,25,25,25,25,25,25
M4S,Central Toronto,Davisville,33,33,33,33,33,33
M4T,Central Toronto,"Moore Park,Summerhill East",3,3,3,3,3,3


## Let's find out how many unique categories can be curated from all the returned venues

In [122]:
print("The unique venues : {}".format(len(venues_df['VenueCategory'].unique())))

The unique venues : 234


# 15. Analysing each Area

In [135]:
toronto_OneHot1=pd.get_dummies(venues_df[['VenueCategory']],prefix="",prefix_sep="")


#Adding columns back to this new data frame.

toronto_OneHot1['PostalCode']=venues_df['PostalCode']
toronto_OneHot1['Borough']=venues_df['Borough']
toronto_OneHot1['Neighborhoods']=venues_df['Neighborhood']


#moving the recently added columns to the starting index

fixed_column=list(toronto_OneHot1.columns[-3:])+list(toronto_OneHot1.columns[:-3])
toronto_OneHot1=toronto_OneHot1[fixed_column]

toronto_OneHot1.head()


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

## Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [136]:
toronto_grouped = toronto_OneHot1.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(38, 237)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,East Toronto,The Beaches,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000

### Now let's create the new dataframe and display the top 10 venues for each PostalCode

In [144]:
num_top_venues=10


indicators=['st','nd','rd']


#Now adding the suffixes

areaColumns=['PostalCode','Borough','Neighborhoods']
frequency=[]

for ind in np.arange(num_top_venues):
    try:
        frequency.append('{}{} Most common Venue'.format(ind+1,indicators[ind]))
    except:
        frequency.append('{}th Most common Venue'.format(ind+1))
        
        
columns=areaColumns+frequency


#create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']


for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

    
# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(38, 13)


,PostalCode,Borough,Neighborhoods,1st Most common Venue,2nd Most common Venue,3rd Most common Venue,4th Most common Venue,5th Most common Venue,6th Most common Venue,7th Most common Venue,8th Most common Venue,9th Most common Venue,10th Most common Venue
0,M4E,East Toronto,The Beaches,Health Food Store,Trail,Pub,Neighborhood,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,M4K,East Toronto,"The Danforth West,Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Brewery,Bubble Tea Shop,Restaurant,Café
2,M4L,East Toronto,"The Beaches West,India Bazaar",Sandwich Place,Gym,Pub,Liquor Store,Burger Joint,Fast Food Restaurant,Burrito Place,Fish & Chips Shop,Italian Restaurant,Food & Drink Shop
3,M4M,East Toronto,Studio District,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Yoga Studio,Chinese Restaurant,Stationery Store,Middle Eastern Restaurant,Clothing Store
4,M4N,Central Toronto,Lawrence Park,Park,Bus Line,Swim School,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
5,M4P,Central Toronto,Davisville North,Convenience Store,Food & Drink Shop,Park,Gym,Breakfast Spot,Sandwich Place,Hotel,Clothing Store,Donut Shop,Dumpling Restaurant
6,M4R,Central Toronto,North Toronto West,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Salon / Barbershop,Burger Joint,Restaurant,Spa,Rental Car Location,Café
7,M4S,Central Toronto,Davisville,Dessert Shop,Pizza Place,Sandwich Place,Coffee Shop,Sushi Restaurant,Italian Restaurant,Café,Gym,Toy / Game Store,Fried Chicken Joint
8,M4T,Central Toronto,"Moore Park,Summerhill East",Playground,Park,Restaurant,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",Coffee Shop,Pub,Restaurant,Sports Bar,Bagel Shop,Supermarket,Sushi Restaurant,Liquor Store,Fried Chicken Joint,Light Rail Station


# 16. Cluster Areas

In [146]:
#Clusteing by using 5 kmeans clusters

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 3, 0, 0, 0, 2, 0])

In [160]:
toronto_merg=toronto_grouped2.copy()



# add clustering labels
toronto_merg["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merg = toronto_merg.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merg.shape)
toronto_merg.head() # check the last columns!

(38, 16)


,PostalCode,Borough,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most common Venue,2nd Most common Venue,3rd Most common Venue,4th Most common Venue,5th Most common Venue,6th Most common Venue,7th Most common Venue,8th Most common Venue,9th Most common Venue,10th Most common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Neighborhood,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Brewery,Bubble Tea Shop,Restaurant,Café
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Sandwich Place,Gym,Pub,Liquor Store,Burger Joint,Fast Food Restaurant,Burrito Place,Fish & Chips Shop,Italian Restaurant,Food & Drink Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Yoga Studio,Chinese Restaurant,Stationery Store,Middle Eastern Restaurant,Clothing Store
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Bus Line,Swim School,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [161]:
#sort the values

toronto_merg.sort_values(["Cluster Labels"],inplace=True)
toronto_merg

,PostalCode,Borough,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most common Venue,2nd Most common Venue,3rd Most common Venue,4th Most common Venue,5th Most common Venue,6th Most common Venue,7th Most common Venue,8th Most common Venue,9th Most common Venue,10th Most common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Neighborhood,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
20,M5K,Downtown Toronto,"Design Exchange,Toronto Dominion Centre",43.647177,-79.381576,0,Coffee Shop,Café,Hotel,Restaurant,Bar,Bakery,Gastropub,Deli / Bodega,Italian Restaurant,American Restaurant
21,M5L,Downtown Toronto,"Commerce Court,Victoria Hotel",43.648198,-79.379817,0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Italian Restaurant,Steakhouse,Deli / Bodega,Seafood Restaurant
24,M5R,Central Toronto,"The Annex,North Midtown,Yorkville",43.672710,-79.405678,0,Café,Coffee Shop,Sandwich Place,Pizza Place,Pharmacy,Indian Restaurant,Pub,BBQ Joint,Jewish Restaurant,Cosmetics Shop
25,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049,0,Café,Bookstore,Bar,Japanese Restaurant,Sandwich Place,Restaurant,Bakery,Chinese Restaurant,Pub,Dessert Shop
26,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market",43.653206,-79.400049,0,Café,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Vietnamese Restaurant,Chinese Restaurant,Dumpling Restaurant,Bakery,Coffee Shop,Farmers Market
27,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.628947,-79.394420,0,Airport Lounge,Airport Service,Airport Terminal,Bar,Airport,Airport Food Court,Harbor / Marina,Sculpture Garden,Boutique,Boat or Ferry
19,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752,0,Coffee Shop,Aquarium,Hotel,Café,Brewery,Scenic Lookout,Fried Chicken Joint,Music Venue,Plaza,Baseball Stadium
28,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Hotel,Seafood Restaurant,Beer Bar,Cocktail Bar,Fast Food Restaurant,Gym
30,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Café,Grocery Store,Park,Nightclub,Convenience Store,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Diner


## Finally, let's visualize the resulting clusters

In [163]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merg['Latitude'], toronto_merg['Longitude'], toronto_merg['PostalCode'], toronto_merg['Borough'], toronto_merg['Neighborhoods'], toronto_merg['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 17. Examine Clusters

## Cluster 1

In [164]:
toronto_merg.loc[toronto_merg['Cluster Labels'] == 0, toronto_merg.columns[[1] + list(range(5, toronto_merg.shape[1]))]]

,Borough,Cluster Labels,1st Most common Venue,2nd Most common Venue,3rd Most common Venue,4th Most common Venue,5th Most common Venue,6th Most common Venue,7th Most common Venue,8th Most common Venue,9th Most common Venue,10th Most common Venue
0,East Toronto,0,Health Food Store,Trail,Pub,Neighborhood,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
20,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Bar,Bakery,Gastropub,Deli / Bodega,Italian Restaurant,American Restaurant
21,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Italian Restaurant,Steakhouse,Deli / Bodega,Seafood Restaurant
24,Central Toronto,0,Café,Coffee Shop,Sandwich Place,Pizza Place,Pharmacy,Indian Restaurant,Pub,BBQ Joint,Jewish Restaurant,Cosmetics Shop
25,Downtown Toronto,0,Café,Bookstore,Bar,Japanese Restaurant,Sandwich Place,Restaurant,Bakery,Chinese Restaurant,Pub,Dessert Shop
26,Downtown Toronto,0,Café,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Vietnamese Restaurant,Chinese Restaurant,Dumpling Restaurant,Bakery,Coffee Shop,Farmers Market
27,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Bar,Airport,Airport Food Court,Harbor / Marina,Sculpture Garden,Boutique,Boat or Ferry
19,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Brewery,Scenic Lookout,Fried Chicken Joint,Music Venue,Plaza,Baseball Stadium
28,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Hotel,Seafood Restaurant,Beer Bar,Cocktail Bar,Fast Food Restaurant,Gym
30,Downtown Toronto,0,Café,Grocery Store,Park,Nightclub,Convenience Store,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Diner


## Cluster 2

In [165]:
toronto_merg.loc[toronto_merg['Cluster Labels'] == 1, toronto_merg.columns[[1] + list(range(5, toronto_merg.shape[1]))]]

,Borough,Cluster Labels,1st Most common Venue,2nd Most common Venue,3rd Most common Venue,4th Most common Venue,5th Most common Venue,6th Most common Venue,7th Most common Venue,8th Most common Venue,9th Most common Venue,10th Most common Venue
22,Central Toronto,1,Garden,Home Service,Yoga Studio,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


## Cluster 3

In [166]:
toronto_merg.loc[toronto_merg['Cluster Labels'] == 2, toronto_merg.columns[[1] + list(range(5, toronto_merg.shape[1]))]]

,Borough,Cluster Labels,1st Most common Venue,2nd Most common Venue,3rd Most common Venue,4th Most common Venue,5th Most common Venue,6th Most common Venue,7th Most common Venue,8th Most common Venue,9th Most common Venue,10th Most common Venue
8,Central Toronto,2,Playground,Park,Restaurant,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
10,Downtown Toronto,2,Park,Playground,Trail,Building,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


## Cluster 4

In [167]:
toronto_merg.loc[toronto_merg['Cluster Labels'] == 3, toronto_merg.columns[[1] + list(range(5, toronto_merg.shape[1]))]]

,Borough,Cluster Labels,1st Most common Venue,2nd Most common Venue,3rd Most common Venue,4th Most common Venue,5th Most common Venue,6th Most common Venue,7th Most common Venue,8th Most common Venue,9th Most common Venue,10th Most common Venue
4,Central Toronto,3,Park,Bus Line,Swim School,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


## Cluster 5

In [168]:
toronto_merg.loc[toronto_merg['Cluster Labels'] == 4, toronto_merg.columns[[1] + list(range(5, toronto_merg.shape[1]))]]

,Borough,Cluster Labels,1st Most common Venue,2nd Most common Venue,3rd Most common Venue,4th Most common Venue,5th Most common Venue,6th Most common Venue,7th Most common Venue,8th Most common Venue,9th Most common Venue,10th Most common Venue
23,Central Toronto,4,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space



Observations:
    
Most of the neighborhoods fall into Cluster 1 which are mostly business areas with cafe, restaurants, supermarkets etc. Cluster 2 is just a garden, Cluster 3 are playground and park, Cluster 4 park and swim school, and lastly Cluster 5 park and trail.